## 作業目標: 使用Jieba進行各項的斷詞操作

這份作業我們會使用中文飯店評論資料集來作為斷詞練習。 [資料集:ChnSentiCorp_htl_all](https://github.com/SophonPlus/ChineseNlpCorpus)

### 讀入文本資料

In [1]:
import pandas as pd

# hint: 可利用pandas讀取CSV
pd_corpus=pd.read_csv('./D06_ChnSentiCorp_htl_all.csv')

print(pd_corpus.head(5))

   label                                             review
0      1  距离川沙公路较近,但是公交指示不对,如果是"蔡陆线"的话,会非常麻烦.建议用别的路线.房间较...
1      1                       商务大床房，房间很大，床有2M宽，整体感觉经济实惠不错!
2      1         早餐太差，无论去多少人，那边也不加食品的。酒店应该重视一下这个问题了。房间本身很好。
3      1  宾馆在小街道上，不大好找，但还好北京热心同胞很多~宾馆设施跟介绍的差不多，房间很小，确实挺小...
4      1               CBD中心,周围没什么店铺,说5星有点勉强.不知道为什么卫生间没有电吹风


In [2]:
#確認所有留言,正評價(label=1)與負評價數量(label=0)
pd_negative, pd_positive = [pd_corpus[pd_corpus['label'] == i] for i in range(2)]

print(f'Total: {len(pd_corpus)}, Positive: {len(pd_positive)}, Negative: {len(pd_negative)}')

Total: 7766, Positive: 5322, Negative: 2444


In [3]:
#移除缺失值
pd_corpus.dropna(inplace=True)

### 建構結巴斷詞Function

建構可將讀入的Pandas DataFrame的文本資料，外加一欄名為`cut`的review欄位斷詞結果

In [4]:
import warnings #忽略警告
warnings.filterwarnings('ignore')

In [5]:
import jieba
jieba.setLogLevel(jieba.logging.INFO)

import paddle
paddle.enable_static()

In [6]:
class JiebaCutingClass(object):
    '''Class to use jeiba to parse corpus from dataframe and cut the corpus
    
    Parameters
    -----------
    key_to_cut: str
        the dataframe key to parse the sentence for jieba cutting
    dic: str
        the dictionary to use for jieba, default is None (use default dictionary)
    userdict: str
        the user defined dictionary to use for jieba, default is None
    '''
    
    def __init__(self, key_to_cut:str, dic:str=None, userdict:str=None):
        
        if dic is not None:
            jieba.set_dictionary(dic)
        
        if userdict is not None:
            jieba.load_userdict(userdict)
        
        self.key_to_cut = key_to_cut
        
        #將paddle 功能開啟
        jieba.enable_paddle()
        
    @staticmethod
    def cut_single_sentence(sentence, use_paddle=False, use_full=False, use_search=False):
        
        if use_search:
            # hint:使用收尋引擎模式進行斷詞
            out = jieba.cut_for_search(sentence)
        else:
            # hint:非收尋引擎模式斷詞，請注意有精確模式、全模式與paddle模式
            out = jieba.cut(sentence, use_paddle=use_paddle, cut_all=use_full)
        return out
            
    def cut_corpus(self, corpus: pd.DataFrame, mode: str) -> pd.DataFrame:
        '''Method to read and cut sentence from dataframe and append another column named cut
        
        Paremeters
        --------------
        corpus: pd.DataFrame
            Input corpus in dataframe
        mode: str
            Jieba mode to be used
        
        Return
        ----------------
        out: pd.Dataframe
            Output corpus in dataframe
        '''
        
        # checking valid mode
        if mode not in ['paddle', 'full', 'precise', 'search']:
            raise TypeError(f'only support `paddle`, `full`, `precise`, and `search` mode, but get {mode}')
            
        # cut the corpus based on mode
        # hint: 根據mode來呼叫相對應的斷詞method
        dict_mode={'paddle': self._paddle_cut(corpus),
                   'full': self._full_cut(corpus),
                   'precise': self._precise_cut(corpus),
                   'search': self._search_cut(corpus)}
        return dict_mode[mode]

    # paddle mode
    def _paddle_cut(self, corpus):
        # hint:將讀進的文本進行斷詞，並將結果append回out的陣列中
        corpus['cut'] = [[word for word in JiebaCutingClass.cut_single_sentence(single_review, use_paddle = True)]\
                         for single_review in corpus[self.key_to_cut]]
        return corpus
    
    # full mode 
    def _full_cut(self, corpus):
        # hint:將讀進的文本進行斷詞，並將結果append回out的陣列中
        corpus['cut'] = [[word for word in JiebaCutingClass.cut_single_sentence(single_review, use_full = True)]\
                         for single_review in corpus[self.key_to_cut]]
        return corpus
    
    # precise mode 
    def _precise_cut(self, corpus):
        # hint:將讀進的文本進行斷詞，並將結果append回out的陣列中
        corpus['cut'] = [[word for word in JiebaCutingClass.cut_single_sentence(single_review)]\
                         for single_review in corpus[self.key_to_cut]]
        return corpus
    
    # search mode
    def _search_cut(self, corpus):
        # hint:將讀進的文本進行斷詞，並將結果append回out的陣列中
        corpus['cut'] = [[word for word in JiebaCutingClass.cut_single_sentence(single_review, use_search = True)]\
                         for single_review in corpus[self.key_to_cut]]
        return corpus

### 使用建構好的斷詞物件對文本進行斷詞

In [1]:
### 請使用精確模式與預設字典對文本進行斷詞

# hint:請先實例化JiebaCutingClass，再進行斷詞
jieba_cut = JiebaCutingClass(key_to_cut='review')
pd_cut = jieba_cut.cut_corpus(pd_corpus.loc[:50, :], mode='precise') #為了避免處理時間過久, 這裡我們只使用前50個進行斷詞

print(pd_cut.head())

Paddle enabled successfully......


   label                                             review  \
0      1  距离川沙公路较近,但是公交指示不对,如果是"蔡陆线"的话,会非常麻烦.建议用别的路线.房间较...   
1      1                       商务大床房，房间很大，床有2M宽，整体感觉经济实惠不错!   
2      1         早餐太差，无论去多少人，那边也不加食品的。酒店应该重视一下这个问题了。房间本身很好。   
3      1  宾馆在小街道上，不大好找，但还好北京热心同胞很多~宾馆设施跟介绍的差不多，房间很小，确实挺小...   
4      1               CBD中心,周围没什么店铺,说5星有点勉强.不知道为什么卫生间没有电吹风   

                                                 cut  
0  [距离, 川沙, 公路, 较近, ,, 但是, 公交, 指示, 不, 对, ,, 如果, 是...  
1  [商务, 大床, 房, ，, 房间, 很大, ，, 床有, 2M, 宽, ，, 整体, 感觉...  
2  [早餐, 太, 差, ，, 无论, 去, 多少, 人, ，, 那边, 也, 不加, 食品, ...  
3  [宾馆, 在, 小, 街道, 上, ，, 不大, 大好, 不大好, 找, ，, 但, 还好,...  
4  [CBD, 中心, ,, 周围, 什么, 没什么, 店铺, ,, 说, 5, 星, 有点, ...  


### 載入繁中字典為指定的字詞進行斷詞

In [8]:
test_string = '我愛cupoy自然語言處理馬拉松課程'
jieba_cut = JiebaCutingClass(key_to_cut='', dic='./D06 jieba_extra_dict/dict.txt.big')

out_string = jieba_cut.cut_single_sentence(test_string, use_full=True) #全模式
print(f'全模式: {[string for string in out_string]}')

out_string = jieba_cut.cut_single_sentence(test_string, use_search=True) #搜尋模式
print(f'搜尋模式: {[string for string in out_string]}')

out_string = jieba_cut.cut_single_sentence(test_string) #精確模式
print(f'精確模式: {[string for string in out_string]}')

out_string = jieba_cut.cut_single_sentence(test_string, use_paddle=True) #paddle 模式
print(f'Paddle模式: {[string for string in out_string]}')

Paddle enabled successfully......


全模式: ['我', '愛', 'cupoy', '自然', '自然語言', '語言', '處理', '馬拉', '馬拉松', '課程']
搜尋模式: ['我', '愛', 'cupoy', '自然', '語言', '自然語言', '處理', '馬拉', '馬拉松', '課程']
精確模式: ['我', '愛', 'cupoy', '自然語言', '處理', '馬拉松', '課程']
Paddle模式: ['我', '愛', 'cupoy', '自然', '語言', '處理', '馬拉松', '課程']
